Write a query to calculate the year-on-year growth rate for the total spend of each product, grouping the results by product ID.

In [ ]:
WITH cte AS (
  SELECT EXTRACT(year from transaction_date) as year
        , product_id
        , SUM(spend) as curr_year_spend
  FROM user_transactions
  GROUP BY 1, 2
  ORDER BY 2, 1
),
spend_variance AS (
  SELECT *
        , LAG(curr_year_spend, 1) OVER(
              PARTITION BY product_id
              ORDER BY year, product_id) as prev_year_spend
  FROM cte)

SELECT *
  , ROUND((curr_year_spend / prev_year_spend - 1) * 100, 2) as yoy_change
FROM spend_variance;

Amazon wants to maximize the number of items it can stock in a 500,000 square feet warehouse. It wants to stock as many prime items as possible, and afterwards use the remaining square footage to stock the most number of non-prime items.

Write a query to find the number of prime and non-prime items that can be stored in the 500,000 square feet warehouse. 

In [ ]:
WITH summary AS (  
  SELECT DISTINCT(item_type)
        , COUNT(item_id) as item_count
        , SUM(square_footage) as total_sf
  FROM inventory
  GROUP BY 1
),  
prime_space AS (
  SELECT *
      , FLOOR(500000 / total_sf) as prime_eligible_space
      , FLOOR(500000 / total_sf) * item_count as prime_eligible_count
  FROM summary
  WHERE item_type = 'prime_eligible'
)

SELECT item_type
      , CASE
          WHEN item_type = 'prime_eligible'
            THEN (floor(500000 / total_sf) * item_count)
          WHEN item_type = 'not_prime'
            THEN FLOOR((500000 -
              (SELECT prime_eligible_space * total_sf FROM prime_space))
              / total_sf)
              * item_count
        END AS item_count
FROM summary
ORDER BY item_type desc;

Your team at JPMorgan Chase is soon launching a new credit card. You are asked to estimate how many cards you'll issue in the first month.

Before you can answer this question, you want to first get some perspective on how well new credit card launches typically do in their first month.

Write a query that outputs the name of the credit card, and how many cards were issued in its launch month.

In [ ]:
WITH cte AS (
  SELECT card_name
        , issued_amount
        , MAKE_DATE(issue_year, issue_month, 1) as date
        , MIN(MAKE_DATE(issue_year, issue_month, 1)) OVER(PARTITION BY card_name) as launch_date
  FROM monthly_cards_issued
)

SELECT card_name
      , issued_amount
FROM cte
WHERE date = launch_date
ORDER BY issued_amount desc;

A phone call is considered an international call when the person calling is in a different country than the person receiving the call.

What percentage of phone calls are international?

In [ ]:
SELECT  
      ROUND(
      100.0 * SUM(CASE WHEN pi.country_id <> pi2.country_id THEN 1 END)
        / COUNT(*), 1) as international_calls_pct
FROM phone_calls pc
JOIN phone_info pi
  ON pc.caller_id = pi.caller_id
JOIN phone_info pi2
  ON pc.receiver_id = pi2.caller_id;

Given a table of tweet data over a specified time period, calculate the 3-day rolling average of tweets for each user. Output the user ID, tweet date, and rolling averages rounded to 2 decimal places.

In [ ]:
SELECT user_id
      , tweet_date
      , ROUND(AVG(tweet_count) OVER(
          PARTITION BY user_id 
          ORDER BY tweet_date 
          ROWS BETWEEN 2 PRECEDING AND CURRENT ROW)
        , 2) as rolling_3d_avg
FROM tweets;